## Mean Shift and Cam Shift Tracking

Mean shift and cam shift methods are most basic methods of tracking.

**Mean Shift**

* Imagine if  we have a set of points and we wanted to assign them into clusters.

* Then we take all our data points and stack red and blue points on them.

* The direction to the closest cluster centroid is determined by where most of the points nearby at.

* So each iteration each blue point will move closer to where the most points are at, which is or will lead to the cluster center.

**Drawback of meanshift**

It won't always detect clusters properly, what may be more reasonable

**How Mean shift is helpful in object tracking**

Mean Shift can be given a target to track, calculate the color histogram of the target area, and then keep sliding the tracking window to the tracking window to the closest match(the cluster center)

**Just using Mean Shift won't change the window size if the target moves away or towards the camera.**

* In that case we can use CAM Shift to update the size of the window.


### Object Tracking using Mean Shift

In [1]:
# Import all the needed modules
import cv2 
import numpy as np

In [2]:
# Capture the webcam frames

cap=cv2.VideoCapture(0)

# first frame
ret,frame1=cap.read()

# In our case we will detect the face and then track it 
face_cascade=cv2.CascadeClassifier('C:/Users/DELL/Desktop/Computer-Vision-with-OpenCV-and-Deep-Learning/Object-Detection-with-OpenCV/haarcascades/haarcascade_frontalface_default.xml')

face_img=frame1.copy()
face_rects=face_cascade.detectMultiScale(face_img)

# We want to track only a single face 
(face_x,face_y,w,h)=tuple(face_rects[0])

trac_window=(face_x,face_y,w,h)
    

# Our region of interest
roi=face_img[face_y : face_y+h, face_x : face_x+w]

# Convert roi's color scale to hsv

hsv_roi=cv2.cvtColor(roi,cv2.COLOR_BGR2HSV)

# For back tracking our window we have to first grab the color histogram of the roi

roi_hist=cv2.calcHist([hsv_roi],[0],None,[180],[0,180])

# Normalise the hist roi 
roi_hist=cv2.normalize(roi_hist,roi_hist,0,255,cv2.NORM_MINMAX)

# Our termination criteria 
term_criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_EPS,10,1)

while True:
    
    ret,frame=cap.read()
    
    # Convert the farme scheme to hsv scale
    hsv=cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
    
    # Back tracking our window
    dst=cv2.calcBackProject([hsv],[0],roi_hist,[0,180],1)
    
    # Track detected face window
    ret,track_window=cv2.meanShift(dst,trac_window,term_criteria)
    
    x,y,w,h=track_window
    
    img2=cv2.rectangle(frame,(x,y),(x+w,y+h),(0,0,255),5)
    
    cv2.imshow('img',img2)
    
    k=cv2.waitKey(1) & 0xFF
    
    if k==27:
        break
        
cap.release()
cv2.destroyAllWindows()





## Object Tracking with CAM Shift

In [2]:
cap=cv2.VideoCapture(0)

ret,frame1=cap.read()

face_cascade=cv2.CascadeClassifier('haarcascades/haarcascade_frontalface_default.xml')


face_rects=face_cascade.detectMultiScale(frame1)

(face_x,face_y,w,h)=tuple(face_rects[0])

track_window=(face_x,face_y,w,h)

roi=frame1[face_y:face_y + h, face_x:face_x +w]

hsv_roi=cv2.cvtColor(roi,cv2.COLOR_BGR2HSV)

roi_hist=cv2.calcHist([hsv_roi],[0],None,[180],[0,180])

cv2.normalize(roi_hist,roi_hist,0,255,cv2.NORM_MINMAX)

term_crit=( cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT , 10, 1)

while True:
    ret,frame=cap.read()
    
    hsv=cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
    
    dst=cv2.calcBackProject([hsv],[0],roi_hist,[0,180],1)
    
    ret1, trackin_window=cv2.CamShift(dst,track_window,term_crit)
    
    pts=cv2.boxPoints(ret1)
    pts=np.int0(pts)
    img2=cv2.polylines(frame,[pts],True,(0,0,255),5)
    
    cv2.imshow('img',img2)
    
    k=cv2.waitKey(20) & 0xFF
    
    if k==27:
        break
        
cv2.destroyAllWindows()
cap.release()
    
    


